In [1]:
import pandas as pd
import numpy as np
import json
import tweepy
import config
from sklearn.preprocessing import MinMaxScaler

In [2]:
df_l = pd.read_csv('./data/labeled_with_features.csv')
df_l.head()

,tweet_id,class_label,text,author_id,created_at,conversation_id,entities,context_annotations,edit_history_tweet_ids,in_reply_to_user_id,attachments,lang,possibly_sensitive,public_metrics,referenced_tweets,reply_settings,source
0,910783670134476800,caution_and_advice,"""Hurricane Maria Live Updates: Catastrophic Fl...",6.279546e+08,2017-09-21 08:32:03+00:00,9.107836701344768e+17,"{'urls': [{'start': 99, 'end': 122, 'url': 'ht...","[{'domain': {'id': '13', 'name': 'Place', 'des...",[910783670134476800],no value,no value,en,no value,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",no value,everyone,IFTTT
1,912134938727780355,rescue_volunteering_or_donation_effort,"Getting food to the island is, obviously, crit...",5.434508e+08,2017-09-25 02:01:30+00:00,9.121349387277804e+17,"{'urls': [{'start': 52, 'end': 75, 'url': 'htt...",no value,[912134938727780355],no value,no value,en,no value,"{'retweet_count': 3, 'reply_count': 0, 'like_c...",[<ReferencedTweet id=912133618675437569 type=q...,everyone,Twitter for iPhone
2,910669838842056704,sympathy_and_support,My heart breaks for the families in Puerto Ric...,3.181771e+09,2017-09-21 00:59:43+00:00,9.106698388420567e+17,"{'annotations': [{'start': 36, 'end': 46, 'pro...","[{'domain': {'id': '13', 'name': 'Place', 'des...",[910669838842056704],no value,no value,en,no value,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",no value,everyone,Twitter for Android
3,912287091026997248,rescue_volunteering_or_donation_effort,"RT @dreynders: #B-FAST sending medical, recons...",3.730165e+09,2017-09-25 12:06:06+00:00,9.122870910269972e+17,"{'hashtags': [{'start': 15, 'end': 17, 'tag': ...","[{'domain': {'id': '29', 'name': 'Events [Enti...",[912287091026997248],no value,no value,en,no value,"{'retweet_count': 15, 'reply_count': 0, 'like_...",[<ReferencedTweet id=912185640690778112 type=r...,everyone,Twitter for iPhone
4,914175720968028161,rescue_volunteering_or_donation_effort,@alan5422 @nytimes U.S. boots have been on the...,2.410331e+08,2017-09-30 17:10:51+00:00,9.141097863414292e+17,"{'hashtags': [{'start': 70, 'end': 81, 'tag': ...","[{'domain': {'id': '47', 'name': 'Brand', 'des...",[914175720968028161],588769770.0,{'media_keys': ['3_914175714278105088']},en,no value,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",[<ReferencedTweet id=911035695581548544 type=q...,everyone,Twitter for iPhone


In [19]:
df = pd.read_csv('labeled_with_features.csv')
df = df.drop_duplicates(subset=['text', 'class_label'], keep=False)  # drop duplicates in dataset
df = df.drop(6322)
df['date'] =  pd.to_datetime(df['created_at'])
df['tweet_id'] = df['tweet_id'].astype(np.int64)
#print(df[df['created_at'].str.len() > 30])
df.head()

,tweet_id,class_label,text,author_id,created_at,conversation_id,entities,context_annotations,edit_history_tweet_ids,in_reply_to_user_id,attachments,lang,possibly_sensitive,public_metrics,referenced_tweets,reply_settings,source,date
0,910783670134476800,caution_and_advice,"""Hurricane Maria Live Updates: Catastrophic Fl...",6.279546e+08,2017-09-21 08:32:03+00:00,9.107836701344768e+17,"{'urls': [{'start': 99, 'end': 122, 'url': 'ht...","[{'domain': {'id': '13', 'name': 'Place', 'des...",[910783670134476800],no value,no value,en,no value,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",no value,everyone,IFTTT,2017-09-21 08:32:03+00:00
1,912134938727780355,rescue_volunteering_or_donation_effort,"Getting food to the island is, obviously, crit...",5.434508e+08,2017-09-25 02:01:30+00:00,9.121349387277804e+17,"{'urls': [{'start': 52, 'end': 75, 'url': 'htt...",no value,[912134938727780355],no value,no value,en,no value,"{'retweet_count': 3, 'reply_count': 0, 'like_c...",[<ReferencedTweet id=912133618675437569 type=q...,everyone,Twitter for iPhone,2017-09-25 02:01:30+00:00
2,910669838842056704,sympathy_and_support,My heart breaks for the families in Puerto Ric...,3.181771e+09,2017-09-21 00:59:43+00:00,9.106698388420567e+17,"{'annotations': [{'start': 36, 'end': 46, 'pro...","[{'domain': {'id': '13', 'name': 'Place', 'des...",[910669838842056704],no value,no value,en,no value,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",no value,everyone,Twitter for Android,2017-09-21 00:59:43+00:00
3,912287091026997248,rescue_volunteering_or_donation_effort,"RT @dreynders: #B-FAST sending medical, recons...",3.730165e+09,2017-09-25 12:06:06+00:00,9.122870910269972e+17,"{'hashtags': [{'start': 15, 'end': 17, 'tag': ...","[{'domain': {'id': '29', 'name': 'Events [Enti...",[912287091026997248],no value,no value,en,no value,"{'retweet_count': 15, 'reply_count': 0, 'like_...",[<ReferencedTweet id=912185640690778112 type=r...,everyone,Twitter for iPhone,2017-09-25 12:06:06+00:00
4,914175720968028161,rescue_volunteering_or_donation_effort,@alan5422 @nytimes U.S. boots have been on the...,2.410331e+08,2017-09-30 17:10:51+00:00,9.141097863414292e+17,"{'hashtags': [{'start': 70, 'end': 81, 'tag': ...","[{'domain': {'id': '47', 'name': 'Brand', 'des...",[914175720968028161],588769770.0,{'media_keys': ['3_914175714278105088']},en,no value,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",[<ReferencedTweet id=911035695581548544 type=q...,everyone,Twitter for iPhone,2017-09-30 17:10:51+00:00


In [20]:
df1 = pd.read_csv('hurricane/hurricane_matthew_2016_train.tsv', delimiter='\t')
df1_1 = pd.read_csv('hurricane/hurricane_matthew_2016_dev.tsv', delimiter='\t')
df1_2 = pd.read_csv('hurricane/hurricane_matthew_2016_test.tsv', delimiter='\t')
df1 = pd.concat([df1, df1_1, df1_2])
df1['hurricane'] = 'matthew'
df1['fatalities'] = 603
df1['damage(billion USD)'] = 16.47
df1.head()

,tweet_id,tweet_text,class_label,hurricane,fatalities,damage(billion USD)
0,783409770493571076,Horrifying. My heart breaks for Haiti. Thinkin...,sympathy_and_support,matthew,603,16.47
1,783683862018818049,Our thoughts and prayers are with those suffer...,sympathy_and_support,matthew,603,16.47
2,784696725285908481,#BreakingNews Hurricane Matthew kills over 800...,injured_or_dead_people,matthew,603,16.47
3,783549594416377856,Southern Haiti effectively cut off after the b...,other_relevant_information,matthew,603,16.47
4,783398732926779392,"Hurricane Matthew hits Haiti, aims at US East ...",other_relevant_information,matthew,603,16.47


In [21]:
df2 = pd.read_csv('hurricane/hurricane_irma_2017_train.tsv', delimiter='\t')
df2_1 = pd.read_csv('hurricane/hurricane_irma_2017_dev.tsv', delimiter='\t')
df2_2 = pd.read_csv('hurricane/hurricane_irma_2017_test.tsv', delimiter='\t')
df2 = pd.concat([df2, df2_1, df2_2])
df2['hurricane'] = 'irma'
df2['fatalities'] = 52
df2['damage(billion USD)'] = 77.16
df2.head()

,tweet_id,tweet_text,class_label,hurricane,fatalities,damage(billion USD)
0,906347001712926721,An inside look at what its like inside a shelt...,displaced_people_and_evacuations,irma,52,77.16
1,908729925905068032,Our parent company @Realogy is matching $75K f...,rescue_volunteering_or_donation_effort,irma,52,77.16
2,907080161513889792,UNICEF is working to bring safe water &amp; sa...,other_relevant_information,irma,52,77.16
3,906964492193595392,Hurricane Irma ripping entire roofs off buildi...,infrastructure_and_utility_damage,irma,52,77.16
4,907227218593685506,Cosmic Kamer on @richardbranson as he reveals ...,infrastructure_and_utility_damage,irma,52,77.16


In [22]:
df3 = pd.read_csv('hurricane/hurricane_maria_2017_train.tsv', delimiter='\t')
df3_1 = pd.read_csv('hurricane/hurricane_maria_2017_test.tsv', delimiter='\t')
df3_2 = pd.read_csv('hurricane/hurricane_maria_2017_dev.tsv', delimiter='\t')
df3 = pd.concat([df3, df3_1, df3_2])
df3['hurricane'] = 'maria'
df3['fatalities'] = 2982
df3['damage(billion USD)'] = 90
df3.head()

,tweet_id,tweet_text,class_label,hurricane,fatalities,damage(billion USD)
0,914134332226330625,San Juan: Trump lashes out with good reason. #...,other_relevant_information,maria,2982,90
1,910783670134476800,Hurricane Maria Live Updates: Catastrophic Flo...,caution_and_advice,maria,2982,90
2,912134938727780355,"Getting food to the island is, obviously, crit...",rescue_volunteering_or_donation_effort,maria,2982,90
3,910669838842056704,My heart breaks for the families in Puerto Ric...,sympathy_and_support,maria,2982,90
4,912287091026997248,"#B-FAST sending medical, reconstruction &amp; ...",rescue_volunteering_or_donation_effort,maria,2982,90


In [23]:
df4 = pd.read_csv('hurricane/hurricane_harvey_2017_train.tsv', delimiter='\t')
df4_1 = pd.read_csv('hurricane/hurricane_harvey_2017_test.tsv', delimiter='\t')
df4_2 = pd.read_csv('hurricane/hurricane_harvey_2017_dev.tsv', delimiter='\t')
df4 = pd.concat([df4, df4_1, df4_2])
df4['hurricane'] = 'harvey'
df4['fatalities'] = 107
df4['damage(billion USD)'] = 125
df4.head()

,tweet_id,tweet_text,class_label,hurricane,fatalities,damage(billion USD)
0,903388763073720321,"Hurricane Harvey killed at least 38 people, bu...",injured_or_dead_people,harvey,107,125
1,901136425348759552,Harvey upped to Category 2 hurricane with 110+...,other_relevant_information,harvey,107,125
2,902853707607535616,A huge shoutout @TexasGuard for all the work y...,rescue_volunteering_or_donation_effort,harvey,107,125
3,902859729701044224,Our thoughts and prayers are with the people h...,sympathy_and_support,harvey,107,125
4,901405975856414720,Homes destroyed on Broadway St in Rockport. Pe...,infrastructure_and_utility_damage,harvey,107,125


In [24]:
df_h = pd.concat([df1, df2, df3, df4], ignore_index=True)
df_h = df_h.rename(columns={'tweet_text' : 'text'})
df_h

,tweet_id,text,class_label,hurricane,fatalities,damage(billion USD)
0,783409770493571076,Horrifying. My heart breaks for Haiti. Thinkin...,sympathy_and_support,matthew,603,16.47
1,783683862018818049,Our thoughts and prayers are with those suffer...,sympathy_and_support,matthew,603,16.47
2,784696725285908481,#BreakingNews Hurricane Matthew kills over 800...,injured_or_dead_people,matthew,603,16.47
3,783549594416377856,Southern Haiti effectively cut off after the b...,other_relevant_information,matthew,603,16.47
4,783398732926779392,"Hurricane Matthew hits Haiti, aims at US East ...",other_relevant_information,matthew,603,16.47
...,...,...,...,...,...,...
27438,902877340191129600,"@realDonaldTrump Meanwhile, #FoxNews was turn ...",injured_or_dead_people,harvey,107,125.00
27439,903055122586054656,Latest from Natl Hurricane Center.. #Harvey is...,other_relevant_information,harvey,107,125.00
27440,902687457375907840,"As we prepare for a new school year, my prayer...",sympathy_and_support,harvey,107,125.00
27441,902606594021744640,Very upsetting to think of the children displa...,displaced_people_and_evacuations,harvey,107,125.00


In [37]:
df_train = df_h.merge(df, on = ['tweet_id', 'class_label'],  how = 'inner')
#df_train.iloc[7235]
df_train = df_train.drop(7235)

,tweet_id,text_x,class_label,hurricane,fatalities,damage(billion USD),text_y,author_id,created_at,conversation_id,...,edit_history_tweet_ids,in_reply_to_user_id,attachments,lang,possibly_sensitive,public_metrics,referenced_tweets,reply_settings,source,date


In [34]:
df_mat = df_train[df_train['hurricane'] == 'matthew']
scaler = MinMaxScaler(feature_range=(0, 100))
df_mat['norm_date'] = scaler.fit_transform(df_mat['date'].values.reshape(-1,1))

df_irma = df_train[df_train['hurricane'] == 'irma']
df_irma['norm_date'] = scaler.fit_transform(df_irma['date'].values.reshape(-1,1))

df_maria = df_train[df_train['hurricane'] == 'maria']
df_maria['norm_date'] = scaler.fit_transform(df_maria['date'].values.reshape(-1,1))

df_harvey = df_train[df_train['hurricane'] == 'harvey']
df_harvey['norm_date'] = scaler.fit_transform(df_harvey['date'].values.reshape(-1,1))

df_new_train = pd.concat([df_mat, df_irma, df_maria, df_harvey], ignore_index=True)
len(df_new_train)

/var/folders/kz/xcrqlgwn7js_lv9xgg0p5c280000gn/T/ipykernel_62157/4007333821.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mat['norm_date'] = scaler.fit_transform(df_mat['date'].values.reshape(-1,1))
/var/folders/kz/xcrqlgwn7js_lv9xgg0p5c280000gn/T/ipykernel_62157/4007333821.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_irma['norm_date'] = scaler.fit_transform(df_irma['date'].values.reshape(-1,1))
/var/folders/kz/xcrqlgwn7js_lv9xgg0p5c280000gn/T/ipykernel_62157/4007333821.py:9: SettingWithC

18786

In [35]:
df_new_train = df_new_train.rename(columns={'text_x' : 'text'})
df_new_train = df_new_train.drop(columns=['text_y'])

In [36]:
df_new_train.to_csv('train_hurricane_with_name.csv')

In [43]:
scaler = MinMaxScaler(feature_range=(0, 100))

df_arthur = pd.read_csv('hurricane/CLEAN_ADD_HUR_DATA/arthur_2014_yn.csv')
df_arthur['hurricane'] = 'arthur'
df_arthur['fatalities'] = 2
df_arthur['damage(billion USD)'] = 0.0395
df_arthur['date'] =  pd.to_datetime(df_arthur['created_at'])
#df_arthur.iloc[208]
df_arthur = df_arthur.drop(208)
df_arthur['norm_date'] = scaler.fit_transform(df_arthur['date'].values.reshape(-1,1))

df_barry = pd.read_csv('hurricane/CLEAN_ADD_HUR_DATA/barry_2019_yn.csv')
df_barry['hurricane'] = 'barry'
df_barry['fatalities'] = 1
df_barry['damage(billion USD)'] = 0.6
df_barry['date'] =  pd.to_datetime(df_barry['created_at'])
df_barry['norm_date'] = scaler.fit_transform(df_barry['date'].values.reshape(-1,1))

df_florence = pd.read_csv('hurricane/CLEAN_ADD_HUR_DATA/florence_2018_yn.csv')
df_florence['hurricane'] = 'florence'
df_florence['fatalities'] = 53
df_florence['damage(billion USD)'] = 24.23
df_florence['date'] =  pd.to_datetime(df_florence['created_at'])
df_florence['norm_date'] = scaler.fit_transform(df_florence['date'].values.reshape(-1,1))

df_ian = pd.read_csv('hurricane/CLEAN_ADD_HUR_DATA/ian_2022_yn.csv')
df_ian['hurricane'] = 'ian'
df_ian['fatalities'] = 157
df_ian['damage(billion USD)'] = 50.2
df_ian['date'] =  pd.to_datetime(df_ian['created_at'])
df_ian['norm_date'] = scaler.fit_transform(df_ian['date'].values.reshape(-1,1))

df_isaias = pd.read_csv('hurricane/CLEAN_ADD_HUR_DATA/isaias_2020_yn.csv')
df_isaias['hurricane'] = 'isaias'
df_isaias['fatalities'] = 18
df_isaias['damage(billion USD)'] = 4.73
df_isaias['date'] =  pd.to_datetime(df_isaias['created_at'])
df_isaias['norm_date'] = scaler.fit_transform(df_isaias['date'].values.reshape(-1,1))

df_nicholas = pd.read_csv('hurricane/CLEAN_ADD_HUR_DATA/nicholas_2021_yn.csv')
df_nicholas['hurricane'] = 'nicholas'
df_nicholas['fatalities'] = 4
df_nicholas['damage(billion USD)'] = 1.1
df_nicholas['date'] =  pd.to_datetime(df_nicholas['created_at'])
df_nicholas['norm_date'] = scaler.fit_transform(df_nicholas['date'].values.reshape(-1,1))

df_sandy = pd.read_csv('hurricane/CLEAN_ADD_HUR_DATA/sandy_2012_yn.csv')
df_sandy['hurricane'] = 'sandy'
df_sandy['fatalities'] = 233
df_sandy['damage(billion USD)'] = 68.7
df_sandy['date'] =  pd.to_datetime(df_sandy['created_at'])
df_sandy['norm_date'] = scaler.fit_transform(df_sandy['date'].values.reshape(-1,1))

In [44]:
df_t = pd.concat([df_arthur, df_barry, df_florence, df_ian, df_isaias, df_nicholas, df_sandy], ignore_index=True)
df_t = df_t.rename(columns={'tweet_text' : 'text'})
df_t

,id,text,author_id,created_at,public_metrics,reply_settings,entities,possibly_sensitive,edit_history_tweet_ids,source,...,conversation_id,in_reply_to_user_id,attachments,withheld,predictions,hurricane,fatalities,damage(billion USD),date,norm_date
0,474324689339559937,@Alexs1776 @NH92276 Still passing around all t...,244172601.0,2014-06-04T22:59:53.000Z,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",everyone,"{'annotations': [{'start': 59, 'end': 66, 'pro...",False,['474324689339559937'],Twitter Web Client,...,4.743240e+17,795680612.0,NaN,NaN,1,arthur,2,0.0395,2014-06-04 22:59:53+00:00,0.011071
1,474324676806995968,#BePrepared Are you familiar with the Saffir-S...,46172460.0,2014-06-04T22:59:50.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",everyone,"{'urls': [{'start': 117, 'end': 139, 'url': 'h...",False,['474324676806995968'],Sprinklr,...,4.743247e+17,NaN,NaN,NaN,1,arthur,2,0.0395,2014-06-04 22:59:50+00:00,0.011029
2,474324671899660288,RT @th3Derek: @Cartoonsbyjosh @richardabetts @...,89165845.0,2014-06-04T22:59:48.000Z,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",everyone,"{'annotations': [{'start': 101, 'end': 117, 'p...",False,['474324671899660288'],Twitter Web Client,...,4.743247e+17,NaN,NaN,NaN,1,arthur,2,0.0395,2014-06-04 22:59:48+00:00,0.011001
3,474324644833398784,RT @Chicagoist: Our music video of the day is ...,10253262.0,2014-06-04T22:59:42.000Z,"{'retweet_count': 2, 'reply_count': 0, 'like_c...",everyone,"{'urls': [{'start': 104, 'end': 126, 'url': 'h...",False,['474324644833398784'],Tweetbot for Mac,...,4.743246e+17,NaN,NaN,NaN,1,arthur,2,0.0395,2014-06-04 22:59:42+00:00,0.010917
4,474324628383744000,RT @Zywave: Live in a #hurricane prone area? H...,302271956.0,2014-06-04T22:59:38.000Z,"{'retweet_count': 2, 'reply_count': 0, 'like_c...",everyone,"{'urls': [{'start': 110, 'end': 132, 'url': 'h...",False,['474324628383744000'],Twitter for iPhone,...,4.743246e+17,NaN,NaN,NaN,1,arthur,2,0.0395,2014-06-04 22:59:38+00:00,0.010861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26326,278255138630680576,"RT @nytimes: An estimated 10,000 jobs lost, 3,...",77364368.0,2012-12-10T21:49:29.000Z,"{'retweet_count': 186, 'reply_count': 0, 'like...",everyone,"{'annotations': [{'start': 78, 'end': 92, 'pro...",False,['278255138630680576'],Twitter for BlackBerry®,...,2.782551e+17,NaN,NaN,NaN,1,sandy,233,68.7000,2012-12-10 21:49:29+00:00,99.991331
26327,278255120695828480,Caught in a hurricane leaving here dead or alive,258360073.0,2012-12-10T21:49:25.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",everyone,NaN,False,['278255120695828480'],Twitter for iPhone,...,2.782551e+17,NaN,NaN,NaN,1,sandy,233,68.7000,2012-12-10 21:49:25+00:00,99.991275
26328,278255081902727168,RT @iamclintmansell: New MANSELL track for '.....,16719058.0,2012-12-10T21:49:15.000Z,"{'retweet_count': 39, 'reply_count': 0, 'like_...",everyone,"{'annotations': [{'start': 122, 'end': 135, 'p...",False,['278255081902727168'],Twitter for iPad,...,2.782551e+17,NaN,NaN,NaN,1,sandy,233,68.7000,2012-12-10 21:49:15+00:00,99.991137
26329,278255071496646656,RT @NJ_Politics: Sen. Menendez visits Long Bea...,491302905.0,2012-12-10T21:49:13.000Z,"{'retweet_count': 2, 'reply_count': 0, 'like_c...",everyone,"{'annotations': [{'start': 22, 'end': 29, 'pro...",False,['278255071496646656'],Twitter for Websites,...,2.782551e+17,NaN,NaN,NaN,1,sandy,233,68.7000,2012-12-10 21:49:13+00:00,99.991110


In [45]:
df_t.to_csv('test_hurricane_with_name.csv')